IMPORT

In [1]:
#Datas & plots
import splitfolders
import wave
from pydub import AudioSegment
from pydub.playback import play
import random
import os
import pandas as pd
import numpy as np
from scipy.io import wavfile
import matplotlib.pyplot as plt


C:\Users\basti\Anaconda3\envs\ProjetE4\lib\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


# Les données

- Utilisation du package splitfolders pour séparer les données en : train_set, validation_set et test_set.
- Récupération des caractéristiques de nos données audios.
- Création d'un csv mis a disposition sur le Drive avec toutes les caractéristiques de chaque fichier.

In [2]:
#Split des données aux labels connus pour créer un train_set,un validation_set et un test_set
splitfolders.ratio('./Data/Known_datas',output="./Data/Train",seed=1337,ratio=(.6,.2,.2))

Copying files: 471 files [00:03, 154.68 files/s]


In [8]:
#Récupération de la fréquence d'échantillonnage (framerate), nombre de channel ou type de son (stéréo/mono), et le bitrate en octet (1 = 8 bit, 2 = 16 bit)
#Méthode pour un random path
def get_random_path():
    dirname = './Data/Known_datas/' + str(random.choice(os.listdir('./Data/Known_datas'))) + str('/')
    random_path = dirname + str(random.choice(os.listdir(dirname)))
    return random_path

def get_infos(path):
    with wave.open(path) as wavefile:
        channels_label = ["Mono","Stéréo"]
        framerate = wavefile.getframerate()
        channel = wavefile.getnchannels()
        sample = wavefile.getsampwidth()
        return {"Name":path,"Fréquence" : str(framerate), "Type": str(channels_label[channel-1]), "BitRate" : str(sample*8)}

audioInfos = get_infos(get_random_path())
print("Fréquence d'échantillonnage = "+audioInfos["Fréquence"])
print("Type de son = "+audioInfos["Type"])
print("BitRate = "+audioInfos["BitRate"])

Fréquence d'échantillonnage = 4000
Type de son = Mono
BitRate = 16


In [5]:
#Csv des caractéristiques pour tous les enregistrements
def create_csv_infos():
    data_list = []
    for dir in os.listdir('./Data/Known_datas/'):
        for file in os.listdir('./Data/Known_datas/'+str(dir)):
            data_list.append(get_infos('./Data/Known_datas/'+str(dir)+"/"+str(file)))
    info_dataframe = pd.DataFrame(data_list)
    info_dataframe.to_csv("./Data/Datas_Infos/known_Datas.csv")

    

In [6]:
#Fonction utile pour display les fichiers audios sous forme graphique.
heart_sound_code_to_name = {
  'Artifact': 'Artifact, noise',
  'Extrahs': 'Extra heart sound',
  'Murmur': 'Murmur',  
  'Normal': 'Normal heartbeat',
}

def show_heart_data(audio_path):
  sample_rate, audio_data = wavfile.read(audio_path, 'rb')
  heart_sound_code = audio_path.split('/')[-2]
  print(f'heart sound name: {heart_sound_code_to_name[heart_sound_code]}')
  print(f'heart sound code: {heart_sound_code}')

  plttitle = f'{heart_sound_code_to_name[heart_sound_code]} ({heart_sound_code})'
  time = np.linspace(0., audio_data.shape[0] / sample_rate,audio_data.shape[0])
  plt.title(plttitle)
  plt.plot(time,audio_data)
  plt.legend()
  plt.xlabel("Time [s]")
  plt.ylabel("Amplitude")
  plt.show()

def play_audio_file(path):
  song = AudioSegment.from_wav(path)
  return song

In [ ]:
random_path = get_random_path()

show_heart_data(random_path)
play(play_audio_file(random_path))

# Le modèle
- Utilisation d'un modèle pré-entrainé pour la reconnaissance audio (méthode de transfert-learning)
- Entrainement de ce modèle sur nos données
- Evaluation du modèle (méthode du Kappa de Cohen)
- Test du modèle


In [13]:
#Import modèle
import tensorflow as tf
import tflite_model_maker as mm
from tflite_model_maker import audio_classifier
import seaborn as sns

import itertools
import glob
import random

from IPython.display import Audio, Image

In [14]:
# Utilisation du modèle pré-entrainé
spec = audio_classifier.YamNetSpec(keep_yamnet_and_custom_heads=True, frame_step=3 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH,frame_length=6 * audio_classifier.YamNetSpec.EXPECTED_WAVEFORM_LENGTH)

INFO:tensorflow:Checkpoints are stored in C:\Users\basti\AppData\Local\Temp\tmp5g02r4m7


### Chargement des train_set, validation_set, test_set

In [ ]:
data_dir = '.\\Data\\Train\\'

train_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'train'), cache=True)

validation_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'val'), cache=True)


test_data = audio_classifier.DataLoader.from_folder(
    spec, os.path.join(data_dir, 'test'), cache=True)